<a href="https://colab.research.google.com/github/wojtekgradzinski/WojtekRepo/blob/main/Copy_of_fashion_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy 3.1

     |████████████████████████████████| 5.9 MB 4.1 MB/s 
     |████████████████████████████████| 10.1 MB 44.3 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 456 kB 74.3 MB/s 
     |████████████████████████████████| 623 kB 47.5 MB/s 
  Created wheel for 3.1: filename=3_1-1.0.0-py3-none-any.whl size=1340 sha256=7d2f42a21887dc6c47ea5576d4b2a6d75c5c61677f3688e1433836b5049e7cc8
  Stored in directory: /root/.cache/pip/wheels/a5/be/1f/0cfa3e1c197209b9c5c097f0e1d8712e240e12713ae61c7345
Successfully built 3.1
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uni

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 777.1 MB 15 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
spacy.__version__

'3.1.3'

In [ ]:
# Import libraries
import spacy
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

nlp = spacy.load("en_core_web_lg")

In [ ]:
#2nd model


with open("/content/drive/MyDrive/Colab Notebooks/7. Natural Language Processing/6. NER/fashion brands.txt") as file:
    dataset = file.read()

doc = nlp(dataset)
print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

Entities: [('Jodie Turner-Smith', 'PERSON'), ('Gucci', 'PERSON'), ('Loewe', 'ORG'), ('Jodie Turner-Smith', 'PERSON'), ('Chanel', 'ORG'), ('Atelier Versace', 'ORG'), ('Diane Kruger', 'PERSON'), ('Alberta', 'GPE'), ('Prada', 'ORG'), ('Jean Paul Gaultier', 'PERSON'), ('Jodie Foster', 'PERSON'), ('H&M', 'ORG'), ('Diodato', 'PERSON'), ('Venetian', 'NORP'), ('Giorgio Armani', 'PERSON'), ('The Golden Lion', 'ORG'), ('Serena Rossi', 'PERSON'), ('Emporio Armani', 'ORG'), ('two', 'CARDINAL'), ('Alba Parietti', 'PERSON'), ('Gessica Notaro', 'PERSON'), ("Wonder Woman's", 'ORG'), ('Paradise Island', 'LOC'), ('Donatella Finocchiaro', 'PERSON'), ('Miu Miu', 'ORG'), ('Giorgio Armani', 'PERSON'), ('Preis', 'ORG'), ('Chanel', 'ORG'), ('Venetian', 'NORP'), ('Janelle Monae', 'PERSON'), ('Cynthia Erivo', 'PERSON'), ('the 91st Academy Awards', 'ORG'), ('Billy Porter', 'PERSON'), ('the 91st Academy Awards', 'ORG'), ('Saoirse Ronan', 'PERSON'), ('Margot Robbie', 'PERSON'), ('Chanel', 'ORG'), ('Emma Stone', 'P

In [ ]:
#2nd model


with open("/content/drive/MyDrive/Colab Notebooks/7. Natural Language Processing/6. NER/fashion brands.txt") as file:
    dataset = file.read()

doc = nlp(dataset)
print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

"""

STEP 1 - TRAIN DATA

"""

# Prepare training data

words = ["Gucci", "Chanel", "Schiaparelli", "Balenciaga",
         "Loewe", "Saint Laurent", "Atelier Versace",
         "Alberta Ferretti", "Lanvin", "Prada", "Jean Paul Gaultier",
         "Givenchy", "Louis Vuitton", "Burberry", "H&M",
         "Armani Privé", "Emporio Armani", "Jayp", "Alexander McQueen", "Calvin Klein"]

train_data = []

with open("/content/drive/MyDrive/Colab Notebooks/7. Natural Language Processing/6. NER/fashion brands.txt") as file:
    dataset = file.readlines()
    for sentence in dataset:
        print("######")
        print("sentence: ", sentence)
        print("######")
        sentence = sentence.lower()
        entities = []
        for word in words:
            word = word.lower()
            if word in sentence:
                start_index = sentence.index(word)
                end_index = len(word) + start_index
                print("word: ", word)
                print("----------------")
                print("start index:", start_index)
                print("end index:", end_index)
                pos = (start_index, end_index,'ORG')
                entities.append(pos)
        element = (sentence.rstrip('\n'), {"entities": entities})

        train_data.append(element)
        print('----------------')
        print("element:", element)

        # ("this is my sentence", {"entities": [0, 4, "PREP"]})
        # ("this is my sentence", {"entities": [6, 8, "VERB"]})

"""

STEP 2 - UPDATE MODEL

"""

ner = nlp.get_pipe("ner")

for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Train model

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(30):
        print("Iteration #", iteration)

        random.shuffle(train_data)
        losses = {}

        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], losses=losses, drop=0.1)
        print("Losses:", losses)



# Save the model

output_dir = Path("/ner/")
nlp.to_disk(output_dir)
print("Saved correctly!")

"""

STEP 3 - TEST THE UPDATED MODEL

"""

print("Loading model...")
nlp_updated = spacy.load(output_dir)

# old sentence, old word
doc = nlp_updated("Cynthia Erivo attends the 92nd Academy Awards, Designer: Versace, Year: 2020")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# old sentence, new word
doc = nlp_updated("Lady Gaga attends the 91st Academy Awards, Designer: Ted Baker, Year: 2019")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# new sentence, new word
doc = nlp_updated("I am a big fan of Tommy Hilfigher")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# new sentence, no word
doc = nlp_updated("Wojtek loves trading crypto!")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

Entities: [('Schiaparelli', 'ORG'), ('Balenciaga', 'ORG'), ('Gucci', 'ORG'), ('Gucci', 'ORG'), ('Gucci', 'ORG'), ('Loewe', 'ORG'), ('Gucci', 'ORG'), ('Chanel', 'ORG'), ('Saint Laurent', 'ORG'), ('Atelier Versace', 'ORG'), ('Armani Privé', 'ORG'), ('Lanvin', 'ORG'), ('Alberta Ferretti', 'ORG'), ('Prada', 'ORG'), ('Jean Paul Gaultier', 'ORG'), ('Givenchy', 'ORG'), ('Louis Vuitton', 'ORG'), ('Burberry', 'ORG'), ('H&M', 'ORG'), ('Armani Privé', 'ORG'), ('Armani Privé', 'ORG'), ('Diodato', 'ORG'), ('Emporio Armani', 'ORG'), ('Emporio Armani', 'ORG'), ('Armani Privé', 'ORG'), ('Jayp', 'ORG'), ('Alexander McQueen', 'ORG'), ('Gucci', 'ORG'), ('Chanel', 'ORG'), ('Alexander McQueen', 'ORG'), ('Calvin Klein', 'ORG'), ('Chanel', 'ORG'), ('Givenchy', 'ORG'), ('Prada', 'ORG'), ('Louis Vuitton', 'ORG'), ('Armani Privé', 'ORG'), ('Calvin Klein', 'ORG'), ('Valentino', 'ORG')]
######
sentence:  Bella Hadid in Schiaparelli couture

######
word:  schiaparelli
----------------
start index: 15
end index: 27

In [ ]:
# new sentence, no word
doc = nlp_updated("Wojtek loves trading crypto!")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

entities: []


In [ ]:
# old sentence, new word
doc = nlp_updated("Lady Gaga attends the 91st Academy Awards, Designer: Ted Baker, Year: 2019")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

entities: []
